In [22]:
def test_has_choice_with_length_gt_1(choices):
    return max([len(x) for x in choices]) > 1

def print_choices_with_number(choices):
    for i, x in enumerate(choices):
        print("{0}. {1}".format(i, x), flush=True)

def ask_for_int():
    int(input())

def ask_for_char(): input()

class Ask():
    def __init__(self, choices=['y', 'n']):
        self.choices = choices
    def ask(self):
        if test_has_choice_with_length_gt_1(self.choices):
            print_choices_with_number()
            ask_res = ask_for_int()
            return self.choices[ask_res]
        else:
            print("/".join(self.choices), flush=True)
            return ask_for_char


class Content():
    def __init__(self,x):
        self.x = x

class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

In [23]:
import unittest

class TestAsk(unittest.TestCase):
    def test_has_choice_with_length_gt_1(self):
        self.assertEqual(test_has_choice_with_length_gt_1(['abc', 'def']), True)

    def test_has_choice_with_length_gt_1_false(self):
        self.assertEqual(test_has_choice_with_length_gt_1(['y', 'n']), False)

unittest.main(argv=[''], verbosity=2, exit=False)

test_has_choice_with_length_gt_1 (__main__.TestAsk) ... ok
test_has_choice_with_length_gt_1_false (__main__.TestAsk) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


In [24]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

In [25]:
class KnowledgeBase():
    def __init__(self, rules):
        self.rules = rules
        self.memory = {}

    def get(self, name):
        if ':'































SyntaxError: expected ':' (394277213.py, line 7)